In [1]:
#import necessary libraries

import cv2 
import os 
import numpy as np


60 FPS é recomendado independentemente da duração do vídeo se o objetivo é analisar movimentos rápidos (como no ténis).

Queremos gerar frames "falsos" (interpolados) entre os reais, de forma uniforme, para alcançar exatamente 143 no total.

A função np.linspace(start, end, num) gera valores uniformemente espaçados entre os números start e end.

In [2]:
vidcap = cv2.VideoCapture("p35_foreflat_s1.avi") #VER MELHOR se posso retirar

def ideal_number_frames(video, file):

    fps = video.get(cv2.CAP_PROP_FPS)  # Extract FPS from metadata
    print(f"FPS of the video: {fps}") #the ideal number of frames per second for fast sports is 60

    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)  # Total of frames
    duracao = total_frames / fps  # duration in seconds

    video.release() # Release of Hardware Resources

    return int(60*duracao)


target_frames = ideal_number_frames(vidcap, "p35_foreflat_s1.avi")

print('Ideal number of total frames to obtain: ' , target_frames)

FPS of the video: 18.0
Ideal number of total frames to obtain:  143


In [5]:
# # Open the video file
vidcap = cv2.VideoCapture("p35_foreflat_s1.avi")

# Create the output directory if it doesn't exist
output_dir = 'data_test'
try:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
except OSError:
    print('Error: Could not create output directory.')

# Read all frames from the video into a list
frames = []
while True:
    success, frame = vidcap.read()
    if not success:
        break
    frames.append(frame)

vidcap.release()

original_frame_count = len(frames)
target_frame_count = target_frames  # desired number of output frames

# Generate evenly spaced positions between 0 and the last frame index
positions = np.linspace(0, original_frame_count - 1, target_frame_count)

# Interpolate frames and save them
for i, pos in enumerate(positions):
    lower = int(np.floor(pos))
    upper = int(np.ceil(pos))

    if lower == upper:
        interpolated = frames[lower]
    else:
        alpha = pos - lower
        interpolated = cv2.addWeighted(
            frames[lower], 1 - alpha,
            frames[upper], alpha,
            0
        )

    filename = os.path.join(output_dir, f"frame_{i:04d}.jpg")
    cv2.imwrite(filename, interpolated)

cv2.destroyAllWindows()

In [6]:
#clean the existed files

def remove_files(folder):
    if not os.path.exists(folder):
        print(f"Error: The folder '{folder}' does not exist!")
        exit()

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        
        try:
            if os.path.isfile(path):  
                os.remove(path)
        except Exception as e:
            print(f"Fail trying to remove {path}: {str(e)}")

remove_files('data_test')